In [42]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
#from sklearn.impute import IterativeImputer
import plotly.express as px

import util
import pickle
from imputation import check_missing, impute_na_with_random, convert_numeric
from pca import pca, create_correlation_matrix
from model import calculate_feature_importances, train_and_evaluate_model
from nn import create_model, train_and_evaluate

In [43]:
df = pd.read_table("../data/drugs.tsv")
df

,CASEID,QUESTID2,CIGEVER,CIGOFRSM,CIGWILYR,CIGTRY,CIGYFU,CIGMFU,CIGREC,CIG30USE,...,II2EMSTY,EMPSTAT4,IIEMPST4,II2EMST4,PDEN00,COUTYP2,MAIIN002,ANALWT_C,VESTR,VEREP
0,1,48694667,1,99,99,19,2012,7,1,1,...,1,4,1,1,2,2,2,4398.40,30017,1
1,2,88530883,1,99,99,14,9999,99,2,93,...,1,4,1,1,1,1,2,1419.19,30052,2
2,3,33251077,1,99,99,14,9999,99,1,2,...,1,99,9,9,3,3,2,14052.62,30028,1
3,4,37814127,1,99,99,16,9999,99,4,93,...,1,4,1,1,1,1,2,10848.18,30055,2
4,5,18762590,1,99,99,14,9999,99,4,93,...,1,1,1,1,2,2,2,5651.73,30013,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55155,55156,13675473,2,99,99,991,9991,91,91,91,...,1,2,1,1,3,3,2,679.36,30003,1
55156,55157,49609908,1,99,99,16,9999,99,4,93,...,1,1,1,1,2,2,2,2296.28,30057,2
55157,55158,81795924,2,99,99,991,9991,91,91,91,...,1,4,1,1,3,3,2,17180.64,30026,1
55158,55159,17198338,2,4,4,991,9991,91,91,91,...,9,99,9,9,1,1,2,3104.06,30051,2


Before we alter any columns, let us see if there are any duplicate entries.
In the cell below, we see the length is the same, therefore we verify each 
Question ID is unique (this is important for ensuring no duplicate entries)

In [44]:
df['QUESTID2'].value_counts().sum() 

55160

## Wrangling and Cleaning

First we will only look at the data where the age of the respondant is between [12, 25]. Then we will assess the missingness of each column by first changing all *'MISSING'* *'SKIPPED'* *'CHOSE NOT TO RESPOND'*, etc values denoted by the Codebook documentation. We will also use vectorized functions to make use of efficient algorithms. 

In [45]:
# only want ages 12 - 25
df = df[(df['CATAGE'] == 1) | (df['CATAGE'] == 2)]
df = df.set_index('CASEID')

Lets also see if we can partition the operations to make the function faster, since we are dealing with a dataset of about a billion observations.
We will attempt to do this through Dask, a distributed systems library, and if it is faster we will apply this later on in the future.

In [46]:
#dask_df = dd.from_pandas(df, npartitions=10)  # You can adjust the number of partitions
# Apply function to each partition
#dask_df = dask_df.map_partitions(lambda df: 
#                                 df.applymap(check_missing)).compute(scheduler='processes')

# Convert back to pandas DataFrame (optional)
#dask_df

Using the % timeit function, we see that there is no significant difference in using Dask. This is probably because our Dataset is large enough to take a while to run on, but not large enough to where it would warrant memory issues. It is in a middle ground where it is large enough to where it would take a while on a local system, but not large enough to benefit from parallizing. 

Moving on, we will use applymap as normal

In [47]:
# make all missing values as pd.NA
df = df.applymap(check_missing)
df

,QUESTID2,CIGEVER,CIGOFRSM,CIGWILYR,CIGTRY,CIGYFU,CIGMFU,CIGREC,CIG30USE,CG30EST,...,II2EMSTY,EMPSTAT4,IIEMPST4,II2EMST4,PDEN00,COUTYP2,MAIIN002,ANALWT_C,VESTR,VEREP
CASEID,,,,,,,,,,,,,,,,,,,,,
1,48694667,1,<NA>,<NA>,19,2012,7,1,1,<NA>,...,1,4,1,1,2,2,2,4398.4,30017,1
2,88530883,1,<NA>,<NA>,14,<NA>,<NA>,2,<NA>,<NA>,...,1,4,1,1,1,1,2,1419.19,30052,2
3,33251077,1,<NA>,<NA>,14,<NA>,<NA>,1,2,<NA>,...,1,<NA>,9,9,3,3,2,14052.62,30028,1
8,35106150,1,<NA>,<NA>,22,<NA>,<NA>,2,<NA>,<NA>,...,1,1,1,1,2,2,2,3211.24,30024,1
9,67182690,1,<NA>,<NA>,16,<NA>,<NA>,1,20,<NA>,...,1,4,1,1,1,1,2,6396.73,30038,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55153,31776420,2,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,1,4,1,1,2,2,2,1649.35,30024,2
55155,10772244,1,<NA>,<NA>,18,<NA>,<NA>,3,<NA>,<NA>,...,1,1,1,1,1,1,2,4594.84,30002,2
55156,13675473,2,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,1,2,1,1,3,3,2,679.36,30003,1


Now, since there are many columns and a lot of NA values, we will drop all columns with over 95% missingness as the amount of data in these columns is not enough in comparison to the rest of the data and may not be a good representation of the overall distribution of these specific features.

In [48]:
df_subset = df.loc[:, df.isna().mean() < 0.95]
df_subset

,QUESTID2,CIGEVER,CIGOFRSM,CIGWILYR,CIGTRY,CIGYFU,CIGMFU,CIGREC,CIG30USE,CIG30AV,...,II2EMSTY,EMPSTAT4,IIEMPST4,II2EMST4,PDEN00,COUTYP2,MAIIN002,ANALWT_C,VESTR,VEREP
CASEID,,,,,,,,,,,,,,,,,,,,,
1,48694667,1,<NA>,<NA>,19,2012,7,1,1,2,...,1,4,1,1,2,2,2,4398.4,30017,1
2,88530883,1,<NA>,<NA>,14,<NA>,<NA>,2,<NA>,<NA>,...,1,4,1,1,1,1,2,1419.19,30052,2
3,33251077,1,<NA>,<NA>,14,<NA>,<NA>,1,2,1,...,1,<NA>,9,9,3,3,2,14052.62,30028,1
8,35106150,1,<NA>,<NA>,22,<NA>,<NA>,2,<NA>,<NA>,...,1,1,1,1,2,2,2,3211.24,30024,1
9,67182690,1,<NA>,<NA>,16,<NA>,<NA>,1,20,3,...,1,4,1,1,1,1,2,6396.73,30038,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55153,31776420,2,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,1,4,1,1,2,2,2,1649.35,30024,2
55155,10772244,1,<NA>,<NA>,18,<NA>,<NA>,3,<NA>,<NA>,...,1,1,1,1,1,1,2,4594.84,30002,2
55156,13675473,2,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,1,2,1,1,3,3,2,679.36,30003,1


Lets see the proportion of NA values in each column

In [49]:
columns_nas = df_subset.isna().sum()/df_subset.shape[0]
columns_nas

QUESTID2    0.062880
CIGEVER     0.000000
CIGOFRSM    0.590585
CIGWILYR    0.590641
CIGTRY      0.619878
              ...   
COUTYP2     0.000000
MAIIN002    0.000000
ANALWT_C    0.070210
VESTR       0.000000
VEREP       0.000000
Length: 1912, dtype: float64

Lets Verify that All Columns of 95% Missingness Have Been Dropped

In [50]:
columns_nas95 = columns_nas[columns_nas > 0.95]
columns_nas95 

Series([], dtype: float64)

Lets get a list of the 'bad' columns. These are columns representing hard drugs (anything other than Tobacco, Alcohol, and Marijuana)

In [51]:
bad_columns = util.get_bad_columns(filepath='../data/Data-Codebook.pdf')

In [52]:
bad_columns

['METHDES',
 'DIETPILS',
 'RITMPHEN',
 'STMCARD',
 'CYLERT',
 'DEXED',
 'DETAMP',
 'DIDREX',
 'ESKAT',
 'IONAMIN',
 'MAZANOR',
 'OBLA',
 'PLEGINE',
 'PRELUDIN',
 'SANOREX',
 'TENUATE',
 'STMNOLST',
 'STIMNEWA',
 'STIMNEWB',
 'STIMNEWC',
 'STIMNEWD',
 'STIMNEWE',
 'STMEVER',
 'STIMAGE',
 'STIMYFU',
 'STIMMFU',
 'STIMREC',
 'STMYRTOT',
 'STTOTFG',
 'STFQFLG',
 'STBSTWAY',
 'STDAYPYR',
 'STDAYPMO',
 'STDAYPWK',
 'METHAGE',
 'METHYFU',
 'METHMFU',
 'METHREC',
 'MTHYRTOT',
 'MTTOTFG',
 'MTFQFLG',
 'MTBSTWAY',
 'MTDAYPYR',
 'MTDAYPMO',
 'MTDAYPWK',
 'COCEVER',
 'COCAGE',
 'COCYFU',
 'COCMFU',
 'COCREC',
 'COCYRTOT',
 'CCTOTFG',
 'CCFQFLG',
 'CCBSTWAY',
 'CCDAYPYR',
 'CCDAYPMO',
 'CCDAYPWK',
 'COCUS30A',
 'CC30EST',
 'CRKEVER',
 'CRKAGE',
 'CRKYFU',
 'CRKMFU',
 'CRAKREC',
 'CRKYRTOT',
 'CRTOTFG',
 'CRFQFLG',
 'CRBSTWAY',
 'CRDAYPYR',
 'CRDAYPMO',
 'CRDAYPWK',
 'CRKUS30A',
 'CR30EST',
 'HEREVER',
 'HERAGE',
 'HERYFU',
 'HERMFU',
 'HERREC',
 'HERYRTOT',
 'HRTOTFG',
 'HRFQFLG',
 'HRBSTWAY',
 'HR

In [53]:
df_subset = df_subset.drop(columns=bad_columns, errors='ignore')
df_subset

,QUESTID2,CIGEVER,CIGOFRSM,CIGWILYR,CIGTRY,CIGYFU,CIGMFU,CIGREC,CIG30USE,CIG30AV,...,II2EMSTY,EMPSTAT4,IIEMPST4,II2EMST4,PDEN00,COUTYP2,MAIIN002,ANALWT_C,VESTR,VEREP
CASEID,,,,,,,,,,,,,,,,,,,,,
1,48694667,1,<NA>,<NA>,19,2012,7,1,1,2,...,1,4,1,1,2,2,2,4398.4,30017,1
2,88530883,1,<NA>,<NA>,14,<NA>,<NA>,2,<NA>,<NA>,...,1,4,1,1,1,1,2,1419.19,30052,2
3,33251077,1,<NA>,<NA>,14,<NA>,<NA>,1,2,1,...,1,<NA>,9,9,3,3,2,14052.62,30028,1
8,35106150,1,<NA>,<NA>,22,<NA>,<NA>,2,<NA>,<NA>,...,1,1,1,1,2,2,2,3211.24,30024,1
9,67182690,1,<NA>,<NA>,16,<NA>,<NA>,1,20,3,...,1,4,1,1,1,1,2,6396.73,30038,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55153,31776420,2,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,1,4,1,1,2,2,2,1649.35,30024,2
55155,10772244,1,<NA>,<NA>,18,<NA>,<NA>,3,<NA>,<NA>,...,1,1,1,1,1,1,2,4594.84,30002,2
55156,13675473,2,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,1,2,1,1,3,3,2,679.36,30003,1


## EDA

After filtering the dataset, we did some preliminary data analysis to find interesting trends.

In [30]:
print(df_subset.columns.to_list())
px.bar(df_subset["GLUE"].value_counts())

['QUESTID2', 'CIGEVER', 'CIGOFRSM', 'CIGWILYR', 'CIGTRY', 'CIGYFU', 'CIGMFU', 'CIGREC', 'CIG30USE', 'CIG30AV', 'CIG30BR2', 'CIG30TPE', 'CIG30MEN', 'CIG30MLN', 'CIG30RO2', 'CIGDLYMO', 'CIGAGE', 'CIG100LF', 'SNFEVER', 'SNUFTRY', 'SNFREC', 'CHEWEVER', 'CHEWTRY', 'CHEWREC', 'SLTTRY', 'SLTREC', 'CIGAREVR', 'CIGARTRY', 'CIGARYFU', 'CIGARMFU', 'CIGARREC', 'CGR30USE', 'CGR30BR2', 'PIPEVER', 'PIPE30DY', 'ALCEVER', 'ALCTRY', 'ALCYFU', 'ALCMFU', 'ALCREC', 'ALCYRTOT', 'ALBSTWAY', 'ALDAYPYR', 'ALDAYPMO', 'ALDAYPWK', 'ALCDAYS', 'NODR30A', 'DR5DAY', 'MJEVER', 'MJAGE', 'MJYFU', 'MJMFU', 'MJREC', 'MJYRTOT', 'MRBSTWAY', 'MRDAYPYR', 'MRDAYPMO', 'MRDAYPWK', 'MJDAY30A', 'AMYLNIT', 'CLEFLU', 'GAS', 'GLUE', 'ETHER', 'SOLVENT', 'LGAS', 'NITOXID', 'SPPAINT', 'AEROS', 'INHNOLST', 'INHEVER', 'INHAGE', 'INHREC', 'IRCIGRC', 'IICIGRC', 'II2CIGRC', 'IRCGRRC', 'IICGRRC', 'II2CGRRC', 'IRPIPLF', 'IIPIPLF', 'IRPIPMN', 'IIPIPMN', 'IRSLTRC', 'IISLTRC', 'II2SLTRC', 'IRCHWRC', 'IICHWRC', 'II2CHWRC', 'IRSNFRC', 'IISNFRC', 'I

HLTINDRG: represents drug abuse

Lets see the value counts for it so we can get an idea of the missingness in the response vector.

In [54]:
y = df_subset['HLTINDRG'] - 1
print(y.value_counts())
print('Number of NA values: ', len(y) - y.value_counts().iloc[0] - y.value_counts().iloc[1])
abuse = px.histogram(y,title="Missingness of HLTINDRG (Drug Abuse?) <br>1: YES 0: NA")
abuse.update_layout(bargap=0.2)
abuse.update_layout(showlegend=False)
abuse

0    8361
1    2584
Name: HLTINDRG, dtype: int64
Number of NA values:  24933


Since we have a large proportion of missing values, we must assess how to handle them.

Data Exclusion: This strategy, also known as listwise or complete-case deletion, involves removing all cases (rows) with missing values. This is the simplest approach and can be used when the number of cases with missing response values is small and are Missing Completely at Random (MCAR). However, you can lose a lot of data if the number of missing values is substantial.

Imputation: You can still use imputation techniques, but need to choose ones that are suitable for binary variables. Common techniques include:

- Mode imputation: Replace the missing values with the mode (most common value) of the non-missing values. This is a simple and fast method, but it can introduce bias if the missing data is not MCAR.

- Predictive mean matching (PMM): This involves using a model to predict the missing values from the other data, and then taking a random draw from the observed values with similar predicted values. This is more robust than simple model-based imputation.

- Logistic regression imputation: Since the response variable is binary, you can fit a logistic regression model using the other variables, and then predict the missing values.

- Multiple Imputation: Similar to the continuous case, multiple imputation can be used. It acknowledges the uncertainty about the imputation by creating several different plausible imputed datasets and combining results obtained from each of them.

Remember, the way you handle missing data in the response variable should depend on the reason why the data is missing. For example, if a response is missing not at random, meaning that the missingness is related to the underlying, unobserved response, excluding these cases or filling in with imputed values can lead to bias in your results. In such cases, more advanced techniques might be necessary, possibly involving using models that can handle missing data directly.

## Imputation

Since we have a lot of missing values, we are going to assess handling them in two different ways:

- listwise deletion: dropping all rows where the response vector value is NA, then sampling from the distribution and imputing to preserve the shape and nature. 
- probabilistic: sampling from the distribution and imputing to preserve the shape and nature in both X and y. 

Then we will compare the distributions between these two, as well as run them through our model to see if there are significant differences in either. 

Our hypothesis is that probabisitic will be a better way of imputing values in our design matrix, X, and that we can just drop rows after this imputation step where the value in our response vector, y are <NA>

Now that we have are data cleaned, imputed, and subsetted to only the columns and rows relevant to our analysis, we can prepare it for PCA. By doing PCA, we want to find the most signifcant feature vectors that explain the variance in the dataset, and ultimately contribute to drug abuse so that we can discover where to prevent drug usage among teens.

In [55]:
X = df_subset.iloc[:, 1:]
X

,CIGEVER,CIGOFRSM,CIGWILYR,CIGTRY,CIGYFU,CIGMFU,CIGREC,CIG30USE,CIG30AV,CIG30BR2,...,II2EMSTY,EMPSTAT4,IIEMPST4,II2EMST4,PDEN00,COUTYP2,MAIIN002,ANALWT_C,VESTR,VEREP
CASEID,,,,,,,,,,,,,,,,,,,,,
1,1,<NA>,<NA>,19,2012,7,1,1,2,<NA>,...,1,4,1,1,2,2,2,4398.4,30017,1
2,1,<NA>,<NA>,14,<NA>,<NA>,2,<NA>,<NA>,<NA>,...,1,4,1,1,1,1,2,1419.19,30052,2
3,1,<NA>,<NA>,14,<NA>,<NA>,1,2,1,112,...,1,<NA>,9,9,3,3,2,14052.62,30028,1
8,1,<NA>,<NA>,22,<NA>,<NA>,2,<NA>,<NA>,<NA>,...,1,1,1,1,2,2,2,3211.24,30024,1
9,1,<NA>,<NA>,16,<NA>,<NA>,1,20,3,112,...,1,4,1,1,1,1,2,6396.73,30038,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55153,2,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,1,4,1,1,2,2,2,1649.35,30024,2
55155,1,<NA>,<NA>,18,<NA>,<NA>,3,<NA>,<NA>,<NA>,...,1,1,1,1,1,1,2,4594.84,30002,2
55156,2,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,1,2,1,1,3,3,2,679.36,30003,1


Dropping all rows where the response vector value is NA, then sampling to impute missing values in X

In [56]:
X_listwise_deletion = df_subset.drop(df_subset[(df_subset['HLTINDRG'] == 94) | (df_subset['HLTINDRG'].isna())].index)
#X_listwise_deletion = X_listwise_deletion.apply(lambda column: column.where(column.notna(), np.random.choice(column.dropna().values, size=len(column))))
X_listwise_deletion # len should be 8361 + 2584 based on y.value_counts()

,QUESTID2,CIGEVER,CIGOFRSM,CIGWILYR,CIGTRY,CIGYFU,CIGMFU,CIGREC,CIG30USE,CIG30AV,...,II2EMSTY,EMPSTAT4,IIEMPST4,II2EMST4,PDEN00,COUTYP2,MAIIN002,ANALWT_C,VESTR,VEREP
CASEID,,,,,,,,,,,,,,,,,,,,,
12,51195804,2,4,4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,1,<NA>,9,9,2,3,2,241.7,30016,2
14,71728934,2,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,1,2,1,1,1,1,2,662.83,30028,1
18,68794044,2,4,4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,1,<NA>,9,9,2,2,2,369.95,30034,2
19,67154315,2,3,3,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,1,<NA>,9,9,1,1,2,1245.3,30024,2
21,41568573,1,<NA>,<NA>,20,<NA>,<NA>,1,21,3,...,1,4,1,1,2,3,2,<NA>,30030,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55137,29379780,2,4,3,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,1,NaN,9,9,1,1,2,1141.87,30038,2
55143,27846250,2,3,3,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,1,NaN,9,9,2,2,2,1429.14,30050,2
55144,10834015,2,4,4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,9,NaN,9,9,1,1,2,738.54,30046,1


Imputing All NA values with values that are from the distribution (this step is harder and takes a longer amount of time)

In [57]:
# Apply the function on each column of df_subset
X_probabilistic = df_subset.apply(impute_na_with_random)
X_probabilistic

,QUESTID2,CIGEVER,CIGOFRSM,CIGWILYR,CIGTRY,CIGYFU,CIGMFU,CIGREC,CIG30USE,CIG30AV,...,II2EMSTY,EMPSTAT4,IIEMPST4,II2EMST4,PDEN00,COUTYP2,MAIIN002,ANALWT_C,VESTR,VEREP
CASEID,,,,,,,,,,,,,,,,,,,,,
1,48694667,1,4,4,19,2012,7,1,1,2,...,1,4,1,1,2,2,2,4398.4,30017,1
2,88530883,1,4,4,14,2011,85,2,30,3,...,1,4,1,1,1,1,2,1419.19,30052,2
3,33251077,1,4,4,14,2011,11,1,2,1,...,1,1,9,9,3,3,2,14052.62,30028,1
8,35106150,1,3,4,22,9989,5,2,5,4,...,1,1,1,1,2,2,2,3211.24,30024,1
9,67182690,1,3,4,16,9989,89,1,20,3,...,1,4,1,1,1,1,2,6396.73,30038,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55153,31776420,2,4,4,12,2012,6,3,5,5,...,1,4,1,1,2,2,2,1649.35,30024,2
55155,10772244,1,4,4,18,9985,1,3,12,3,...,1,1,1,1,1,1,2,4594.84,30002,2
55156,13675473,2,4,4,16,2012,11,1,15,5,...,1,2,1,1,3,3,2,679.36,30003,1


In [58]:
scaler = StandardScaler()
scaled_data = scaler.fit_transform(X_probabilistic)
scaled_data

array([[-0.20608911, -1.27195782,  0.4003776 , ...,  1.67939211,
        -0.76247569, -1.02129816],
       [ 1.35879995, -1.27195782,  0.4003776 , ..., -0.1506416 ,
         1.25156655,  0.97914599],
       [-0.81276081, -1.27195782,  0.4003776 , ...,  7.60967163,
        -0.12949098, -1.02129816],
       ...,
       [-1.58175072,  0.78618959,  0.4003776 , ..., -0.60509558,
        -1.56809258, -1.02129816],
       [-0.17013562, -1.27195782,  0.4003776 , ...,  0.38812682,
         1.53928687,  0.97914599],
       [-1.44336175,  0.78618959,  0.4003776 , ...,  0.88432032,
         1.19402248,  0.97914599]])

## Feature Significance

Now before we choose a model, we need to assess the most important features. Since we are working with mostly categorical columns, PCA will not work well compared to Tree Based Algorithms. To start we have decided to use XGBoost to calculate feature importances.

### XGBoost

Some notes on XGBoost:

- Speed and Performance: comparatively faster than other ensemble classifiers!

- Parallelizable: Because the core XGBoost algorithm is parallelizable, it can harness the power of multicore computers. It is also parallelizable onto GPU's and across networks of computers, making it feasible to train models on very large datasets.

- Wide Variety of Tuning Parameters: XGBoost internally has parameters for cross-validation, regularization, user-defined objective functions, missing values, tree parameters, scikit-learn compatible API, etc.

- Handling Missing Values: XGBoost has an in-built routine to handle missing values. (Our cleaning should have handled this)

- Regularization: XGBoost is also well known for having built-in L1 (Lasso Regression) and L2 (Ridge Regression) regularization which prevents the model from overfitting.

In [59]:
#pca(X=X_probabilistic, scaled_data=scaled_data, thres=.95)
#correlation_matrix = create_correlation_matrix(X_probabilistic)
#correlation_matrix

#most_correlated_features = abs(correlation_matrix).drop(["SUMFLAG"]).sort_values(ascending = False)
#most_correlated_features

In [60]:
# convert response vector to 0s and 1s
X_probabilistic['HLTINDRG'] = X_probabilistic['HLTINDRG'] - 1

# ensure all column dtypes are int / float, not object
X_probabilistic = convert_numeric(X_probabilistic)

# get most important
most_important_features = calculate_feature_importances(df=X_probabilistic, response='HLTINDRG')

/Users/palvins/.pyenv/versions/3.9.4/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


In [61]:
most_important_features[most_important_features['Importance'] > 0.000]

,Feature,Importance
1747,HLTINALC,0.029224
1760,IRPRVHLT,0.018964
1748,HLTINMNT,0.015443
1745,PRVHLTIN,0.013546
725,IICGRGNM,0.004912
...,...,...
1165,AMHINP2,0.000143
169,IICIGFM,0.000136
1679,UADPEOP,0.000118
572,NEDFLG01,0.000077


# Running and Comparing Different Models

Now that we have an idea of the most significant features, we will run and compare different Supervised algorthms to see how they perform on predicting whether a person uses drugs based on the data. Later on we can also run Unsupervised Clustering methods to identify demographics likely to use drugs.

### Neural Networks

Neural networks, including Convolutional Neural Networks (CNNs), can be used with categorical data, but the data needs to be prepared appropriately and the right kind of network architecture must be chosen.

We should consider:

- Embedding Layers: In Neural Networks, categorical data is typically transformed into numeric form using an embedding layer, which maps each category to a dense vector of real numbers. The advantage of this approach is that the network can learn a representation that captures the relationships between categories in a way that one-hot encoding cannot. Our dataset appears to have satisfied this

- Network Architecture: CNNs might not be the best fit for categorical data, *unless* the categorical data has some spatial or sequential structure. Fully connected networks (FCNs) or Multi-layer Perceptrons (MLPs) are usually more appropriate for structured, tabular data.

- Overfitting: Neural networks, particularly deep ones, can overfit to the training data if it is not large or diverse enough. Techniques like regularization, dropout, and early stopping can mitigate this risk.

**Note:** 
- the code for the cel below is not complete, so it is commented out for now. It also takes a very long time to run on a laptop since it is handling many layers and computations within our current Neural Network design. 
- This also suggests a Boosted Decision Tree may be a more efficient approach, and based on [this article](https://towardsdatascience.com/when-and-why-tree-based-models-often-outperform-neural-networks-ceba9ecd0fd8), produce a more accurate result.

In [62]:
categorical_vars = most_important_features['Feature'].tolist()
X = X_probabilistic[categorical_vars]
y = X_probabilistic['HLTINDRG']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


#model, loss, accuracy = train_and_evaluate(categorical_vars, X, y)

#print(f'Test loss: {loss}, Test accuracy: {accuracy}')

### Boosted Decision Trees (using XGBoost)

The boosting technique is powerful, but it can lead to overfitting if not used with care. It's important to tune the parameters, like the learning rate and the number of trees, to get a model that generalizes well.
[Read more here](https://dsc-courses.github.io/dsc140a-2023-wi/materials/lectures/17-boosting/slides.pdf)

In [63]:
# create XGBClassifier
xg_clf = xgb.XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=5)
train_and_evaluate_model(xg_clf, X, y, "xgboost_model", categorical_vars)
pickle.dump(xg_clf, open("../models/xgboost_model.pkl", "wb"))

Confusion Matrix:
[[5383   83]
 [1353  357]]
Accuracy:  0.7998885172798216

Cross-validation scores:
[0.80128205 0.80406912 0.80392977 0.80027875 0.8       ]

Mean cross-validation score:
0.8019119364198899
Feature: HLTINALC, Importance: 0.045717258006334305
Feature: IRPRVHLT, Importance: 0.015555442310869694
Feature: HLTINMNT, Importance: 0.019229477271437645
Feature: PRVHLTIN, Importance: 0.015874357894062996
Feature: IICGRGNM, Importance: 0.003637269837781787
Feature: AMHSVTYP, Importance: 0.003564248327165842
Feature: AMHOUTP3, Importance: 0.002289671218022704
Feature: PRPKCIG2, Importance: 0.0021505586337298155
Feature: FUPSY18, Importance: 0.0
Feature: IIANLFY, Importance: 0.0019843021873384714
Feature: PRBSOLV2, Importance: 0.002465196419507265
Feature: FRDMEVR2, Importance: 0.0015939796576276422
Feature: CDCGMO, Importance: 0.0010191334877163172
Feature: ALCFMFPB, Importance: 0.0
Feature: IRSNFRC, Importance: 0.001981204841285944
Feature: ETHER, Importance: 0.0
Feature: ALCWD2S

### Logistic Regression using Lasso / Ridge regularization

- Lasso Regression (L1 Regularization): This type of regression tends to make the coefficients of less important features zero, effectively eliminating them. This can be useful in feature selection.

- Ridge Regression (L2 Regularization): This type of regression reduces the magnitude of the coefficients but does not make them zero. It helps to reduce the model complexity and multicollinearity.

In [64]:
# Create a Logistic Regression model with L2 regularization
lr = LogisticRegression(penalty='l2')
train_and_evaluate_model(lr, X, y, "logistic_regression_model", categorical_vars)


Confusion Matrix:
[[5466    0]
 [1710    0]]
Accuracy:  0.7617056856187291

Cross-validation scores:
[0.76295987 0.76295987 0.76282051 0.76292683 0.76292683]

Mean cross-validation score:
0.762918780759714


In [65]:
# Create a Random Forest classifier
rf_clf = RandomForestClassifier(n_estimators=100)
train_and_evaluate_model(rf_clf, X, y, "random_forest_model", categorical_vars)

Confusion Matrix:
[[5463    3]
 [1678   32]]
Accuracy:  0.7657469342251951

Cross-validation scores:
[0.76978818 0.76923077 0.77132107 0.76850174 0.77533101]

Mean cross-validation score:
0.7708345549819569
Feature: HLTINALC, Importance: 0.022661995684985614
Feature: IRPRVHLT, Importance: 0.0025652400238287496
Feature: HLTINMNT, Importance: 0.019626853593659545
Feature: PRVHLTIN, Importance: 0.0028894379541724484
Feature: IICGRGNM, Importance: 0.00015011820327674196
Feature: AMHSVTYP, Importance: 0.00039043808388064413
Feature: AMHOUTP3, Importance: 0.0003521232688123079
Feature: PRPKCIG2, Importance: 0.00023221974362297768
Feature: FUPSY18, Importance: 0.000214805906178954
Feature: IIANLFY, Importance: 0.00020360148538177896
Feature: PRBSOLV2, Importance: 0.00039791053860385365
Feature: FRDMEVR2, Importance: 0.00026476680543750396
Feature: CDCGMO, Importance: 0.0001747692512674706
Feature: ALCFMFPB, Importance: 0.0003609713751130582
Feature: IRSNFRC, Importance: 0.00041881147709421987